In [2]:
# ref: 

from hashlib import sha256

class Node:
    def __init__(self, data):
        self.left     = None
        self.right    = None
        self.parent   = None
        self.sibling  = None
        self.position = None
        self.data     = data
        self.hash = sha256(data.encode()).hexdigest()
       
class Tree:
    def __init__(self, leaves):
        self.leaves = [Node(leaf) for leaf in leaves]
        self.layer  = self.leaves[::]
        self.root   = None
        self.build_tree()
    
    def build_layer(self):
        new_layer = []
        
        # 要素数が奇数なら最後の要素を複製して追加
        if len(self.layer) % 2 == 1:
            self.layer.append(self.layer[-1])
        
        for i in range(0, len(self.layer), 2):
            left = self.layer[i]
            right = self.layer[i+1]
            parent = Node(left.hash + right.hash)
            
            left.parent = parent
            left.sibling = right
            left.position = "left"
            
            right.parent = parent
            right.sibling = left
            right.position = "right"
            
            parent.left = left
            parent.right = right
            
            new_layer.append(parent)
        
        self.layer = new_layer
    
    def build_tree(self):
        while len(self.layer) > 1:
            self.build_layer()
        self.root = self.layer[0].hash
    
    def search(self, data):
        target = None
        hash_value = sha256(data.encode()).hexdigest()
        for node in self.leaves:
            if node.hash == hash_value:
                target = node
        return target
    
    def get_pass(self, data):
        target = self.search(data)
        markle_pass = []
        if not(target):
            return
        markle_pass.append(target.hash)
        while target.parent:
            sibling = target.sibling
            markle_pass.append((sibling.hash, sibling.position))
            target = target.parent
        return markle_pass   
      
def caluculator(markle_pass):
    value = markle_pass[0]
    for node in markle_pass[1:]:
        sib = node[0]
        position = node[1]
        if position == "right":
            value = sha256(value.encode() + sib.encode()).hexdigest()
        else:
            value = sha256(sib.encode() + value.encode()).hexdigest()
    return value      

In [14]:
block = ["あめ", "あめ", "みかん", "みかん", "みかん", "りんご", "りんご", "どーなっつ", "どーなっつ"]
print(block)

merkletree = Tree(block)

merkle_leaves = merkletree.leaves
print(merkle_leaves)

merkle_layer = merkletree.layer
print(merkle_layer)

merkle_root = merkletree.root
print(merkle_root)

merkle_pass = merkletree.get_pass("みかん")
print(merkle_pass)

candidate = caluculator(merkle_pass)
print(candidate)

['あめ', 'あめ', 'みかん', 'みかん', 'みかん', 'りんご', 'りんご', 'どーなっつ', 'どーなっつ']
[<__main__.Node object at 0x7f7cba7deb20>, <__main__.Node object at 0x7f7cba7dee50>, <__main__.Node object at 0x7f7cba7decd0>, <__main__.Node object at 0x7f7cba7deb50>, <__main__.Node object at 0x7f7cba7de850>, <__main__.Node object at 0x7f7cba7dec10>, <__main__.Node object at 0x7f7cba7de460>, <__main__.Node object at 0x7f7cba7de4f0>, <__main__.Node object at 0x7f7cba7de610>]
bf03b42cf8e4e8581e08c7595775dc2d9d79c34f9b95885a5d42df100786a218
['1cbe93b936a4bd4d40c2224462023917cd77c962376feb9128d9e569c9856aaf', ('4261abfc91324dc5319312592125610a16b0b0a996fcdfae1d24766b918afae9', 'right'), ('0a6b2aa3a1fc41a9394c48adbf1cb1c3eb3e3a9db975906fb743308f23e1f15b', 'right'), ('2165f2270bbe8ce292395df14cb4e65e271fe608013c018d6b5864ec9f1865c8', 'left'), ('94f4c2ba6d697dc41109e2194343bf721e2bbc5de48068609578c0f4ba5481ed', 'right')]
bf03b42cf8e4e8581e08c7595775dc2d9d79c34f9b95885a5d42df100786a218


In [16]:
block = ["あめ", "あめ", "みかん", "みかん", "りんご", "りんご", "どーなっつ", "どーなっつ"]
print(block)

merkletree = Tree(block)

merkle_pass = merkletree.get_pass("みかん")
print(merkle_pass)

candidate = caluculator(merkle_pass)
print(candidate)

merkle_root = merkletree.root
print(merkle_root)

['あめ', 'あめ', 'みかん', 'みかん', 'りんご', 'りんご', 'どーなっつ', 'どーなっつ']
['1cbe93b936a4bd4d40c2224462023917cd77c962376feb9128d9e569c9856aaf', ('1cbe93b936a4bd4d40c2224462023917cd77c962376feb9128d9e569c9856aaf', 'left'), ('aa280a624da2b88334623a7033f5a6eac1da7012ace335a6366828f59377ed1b', 'left'), ('4d8a97b9474b34c013c92ba1662bb98ae8cab97681ae21b9843db45ee2854730', 'right')]
b8cc63db762632990e49bea7832293ae2d18cff643c78c1e2274c7fcf757354a
b8cc63db762632990e49bea7832293ae2d18cff643c78c1e2274c7fcf757354a
